<a href="https://colab.research.google.com/github/rianaditro/classification_web_app/blob/main/classification_C5_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>